Ch.6 행렬과 벡터 계산
===================

* 벡터 연산 병목 지점의 위치
* 계산 과정 중에 CPU를 효율저긍로 사용하는지 알아볼 수 있는 도구
* 산술 계산에서 numpy가 순수 파이썬보다 더 빠른 이유
* 캐시 미스와 페이지 폴트
* 코드 내에서 메모리 할당을 추적하는 방법

아래코드들은 공부하기 위해 손수 손으로 작성한것이고

프로파일링에 사용된 py파일들은 저자의 파일을 사용하였다.

6.1 문제 소개
-----------

In [2]:
import time

grid_shape = (640,640)

# 한번의 dt이후를 계산하는 함수
def evolve(grid,dt,D=1.0):
    xmax,ymax = grid_shape
    new_grid = [ [0.0] * ymax for x in range(xmax)]   # ---> 640x640 of all elts are 0.0
    for i in range(xmax):
        for j in rnage(ymax):
            grid_xx = (
                grid[(i+1) % xmax][j] + grid[(i-1) % xmax][j] - 2.0 * grid[i][j]
            )
            grid_yy = (
                grid[i][(j+1) % ymax] + grid[i][(j-1) % ymax] - 2.0 * grid[i][j]
            )
            new_grid[i][j] = grid[i][j] + dt*D*(grid_xx + grid_yy)
    return new_grid


def run_experiment(num_iterations):
    # 초기조건 설정
    xmax,ymax = grid_shape
    grid = [ [0.0] * ymax for x in range(xmax)]
    
    # 시뮬레이션 영역의 중간에 물감이 한 방울
    # 떨어진 상태를 시뮬레이션 하기 위한 초기 조건들
    block_low = int(grid_shape[0]*0.4)
    block_high = int(grid_shape[0]*0.5)
    for i in range(block_low,block_high):
        for j in range(block_low,block_high):
            grid[i][j] = 0.005
            
    # 초기 조건을 변경
    start = time.time()
    for i in range(num_iterations):
        grid = evolve(grid,0.1)
    return time.time()-start

In [12]:
import os
os.chdir('/home/jaeheyoungjan/HPP/high_performance_python_2e/06_matrix/diffusion_2d')

In [14]:
!kernprof -lv diffusion_python.py

Wrote profile results to diffusion_python.py.lprof
Timer unit: 1e-06 s

Total time: 594.825 s
File: diffusion_python.py
Function: evolve at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13                                           @profile
    14                                           def evolve(grid, dt, D=1.0):
    15       500        518.0      1.0      0.0      xmax, ymax = grid_shape
    16       500   19317259.0  38634.5      3.2      new_grid = [[0.0 for x in range(grid_shape[1])] for x in range(grid_shape[0])]
    17    320500     165155.0      0.5      0.0      for i in range(xmax):
    18 205120000  100173651.0      0.5     16.8          for j in range(ymax):
    19                                                       grid_xx = (
    20 204800000  168086939.0      0.8     28.3                  grid[(i + 1) % xmax][j] + grid[(i - 1) % xmax][j] - 2.0 * grid[i][j]
    21                                                       )
    22             

Line7 : global namespace에서 가져와야 하므로 실행될 때마다 시간을 많이 잡아먹는다.
    
Line8 : 한번할때 시간이 많이 걸리는것에 비해 전체실행시간으로 보면 비중이 낮다. grid를 키울수록 %Time이 늘것
    
전체적으로 도함수를 계산하고 행렬을 갱신하는데에 시간이 많이 소비되었다.

* [예제 6-6] 메모리 할당을 줄인 순수 파이썬 2차원 확산 방정식 코드

new_grid라는 리스트를 매번 새로 할당하는데에 시간이 많이 걸리므로 이를 수정

---> 메모리 할당은 비용이 많이 든다는 것을 알 수 있다.

In [15]:
import time

grid_shape = (640,640)

# 한번의 dt이후를 계산하는 함수
def evolve(grid,dt,out,D=1.0):
    xmax,ymax = grid_shape
    for i in range(xmax):
        for j in rnage(ymax):
            grid_xx = (
                grid[(i+1) % xmax][j] + grid[(i-1) % xmax][j] - 2.0 * grid[i][j]
            )
            grid_yy = (
                grid[i][(j+1) % ymax] + grid[i][(j-1) % ymax] - 2.0 * grid[i][j]
            )
            out[i][j] = grid[i][j] + dt*D*(grid_xx + grid_yy)



def run_experiment(num_iterations):
    # 초기조건 설정
    xmax,ymax = grid_shape
    next_grid = [ [0.0] * ymax for x in range(xmax)]
    grid = [ [0.0] * ymax for x in range(xmax)]
    
    # 시뮬레이션 영역의 중간에 물감이 한 방울
    # 떨어진 상태를 시뮬레이션 하기 위한 초기 조건들
    block_low = int(grid_shape[0]*0.4)
    block_high = int(grid_shape[0]*0.5)
    for i in range(block_low,block_high):
        for j in range(block_low,block_high):
            grid[i][j] = 0.005
            
    # 초기 조건을 변경
    start = time.time()
    for i in range(num_iterations):
        evolve(grid,0.1,next_grid)
        grid,next_grid = next_grid,grid
    return time.time()-start

In [20]:
!kernprof -lv diffusion_python_memory.py

Wrote profile results to diffusion_python_memory.py.lprof
Timer unit: 1e-06 s

Total time: 429.986 s
File: diffusion_python_memory.py
Function: evolve at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13                                           @profile
    14                                           def evolve(grid, dt, out, D=1.0):
    15       500        278.0      0.6      0.0      xmax, ymax = grid_shape
    16    320500     108716.0      0.3      0.0      for i in range(xmax):
    17 205120000   66310333.0      0.3     15.4          for j in range(ymax):
    18                                                       grid_xx = (
    19 204800000  127199171.0      0.6     29.6                  grid[(i + 1) % xmax][j] + grid[(i - 1) % xmax][j] - 2.0 * grid[i][j]
    20                                                       )
    21                                                       grid_yy = (
    22 204800000  131963217.0      0.6     30.7            

6.3 메모리 단편화
----------------

파이썬은 벡터 연산을 기본으로 제공하지 않는다.

이유 1. 파이썬의 리스트는 실제 데이터를 가리키는 포인터를 저장한다.

이유 2. 파이썬 바이트 코드는 벡터 연산에 최적화되지 않았다

따라서 for 루프는 벡터 연산을 언제 수행해야 도움이 되는지 예측할 수 없다.

ex) grid[5][2]를 실행하면 먼저 grid리스트에서 5번째 항목을 찾아 반환한다. 이 값은 데이터가 실제로 저장된 위치를 가리키는 포인터다. 그리고 이렇게 반환된 객체에서 2번쨰 항목을 찾는다. 이과정까지 거쳐야 실제 데이터가 저장된 위치를 알 수 있다.

In [ ]:
!perf stat -e cycles,instructions,\
    cache-references,cache-misses,branches,brance-misses,task-clock,faults,\
    minor-faults,cs,migrations python diffusion_python_memory.py

#sudo 권한이 없어 결과는 저자의 것을 베껴옴


Performance counter stats for 'python diffusion_python_memory.py':

       415,864,974,126      cycles                    #    2.889 GHz                      
     1,210,522,769,388      instructions              #    2.91  insn per cycle           
           656,345,027      cache-references          #    4.560 M/sec                    
           349,562,390      cache-misses              #   53.259 % of all cache refs      
       251,537,944,600      branches                  # 1747.583 M/sec                    
         1,970,031,461      branch-misses             #    0.78% of all branches          
         143934.730837      task-clock (msec)         #    1.000 CPUs utilized          
                12,791      faults                    #    0.089 K/sec                  
                12,791      minor-faults              #    0.089 K/sec                  
                   117      cs                        #    0.001 K/sec                  
                     6      migrations                #    0.000 K/sec                  
                 
     143.935522122 seconds time elapsed



* numpy예제 --> numpy가 가장 빠르다

(1)에는 vector에 대한 두 번의 루프가 내재되어 있다. 하나는 곱셈을 수행하고 다른 하나는 덧셈을 수행한다. 이 루프는 norm_square_list_comprehension의 루프와 유사하지만 numpy의 최적화된 계산 코드를 사용해 실행된다.

In [31]:
from array import array
import numpy
import time


def norm_square_list(vector):
    #vector = list(range(1000000))
    norm = 0
    for v in vector:
        norm += v * v
    return norm

def norm_square_list_comprehension(vector):
    #vector = list(range(1000000))
    return sum([v*v for v in vector])

def norm_square_array(vector):
    #vector_array = array('l',range(1000000))
    norm = 0
    for v in vector:
        norm += v*v
    return norm

def norm_square_numpy(vector):
    #vector_np = numpy.arange(1000000)
    return numpy.sum(vector*vector)       #--------(1)

def norm_square_numpy_dot(vector):
    #vector_np = numpy.arange(1000000)
    return numpy.dot(vector,vector)


In [33]:
vector = list(range(1000000))
vector_array = array('l',range(1000000))
vector_np = numpy.arange(1000000)
%timeit norm_square_list(vector)
%timeit norm_square_list_comprehension(vector)
%timeit norm_square_array(vector_array)
%timeit norm_square_numpy(vector_np)
%timeit norm_square_numpy_dot(vector_np)

55.8 ms ± 873 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
59.9 ms ± 388 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
64.4 ms ± 308 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.27 ms ± 9.96 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
668 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


6.4 넘파이를 이용한 확산 방정식 해법
-------------------------------------------------

- numpy를 이용해서 순수 파이썬 코드를 벡터화시켜보자.

- numpy의 roll함수 소개

In [7]:
import numpy as np

print(
    np.roll([1,2,3,4],1)
    )

print(
    np.roll([[1,2,3],[4,5,6]],1,axis=1)
    )

[4 1 2 3]
[[3 1 2]
 [6 4 5]]


roll 함수를 사용하여 [예제6-6]의 파이썬 확산 방정식 코드를 수정

* 예제6-9 numpy를 사용한 파이썬 확산 방정식 최초 버전

In [9]:
import time
from numpy import (zeros,roll)

grid_shape = (640,640)

def laplacian(grid):
    return (
        roll(grid,+1,0)+
        roll(grid,-1,0)+
        roll(grid,+1,1)+
        roll(grid,-1,1)-
        4 * grid
            )

def evolve(grid,dt,D=1):
    return grid + dt*D*laplacian(grid)

def run_experiment(num_iterations):
    grid = zeros(grid_shape)
    
    block_row = int(grid_shape[0]*0.4)
    block_high = int(grid_shape[0]*0.5)
    grid[block_row:block_high,block_row:block_high] = 0.005
    
    start = time.time()
    for i in range(num_iterations):
        grid = evolve(grid,0.1)
        
    return time.time() - start

&nbsp;
&nbsp;
&nbsp;
&nbsp;
&nbsp;

* [예제 6-10] numpy를 사용한 2차원 확산 방정식의 성능 측정

(sudo 권한이 없어 결과는 저자의 것을 베껴옴)

In [ ]:
!perf stat -e cycles,instructions,\
    cache-references,cache-misses,branches,brance-misses,task-clock,faults,\
    minor-faults,cs,migrations python diffusion_numpy.py


 Performance counter stats for 'python diffusion_numpy.py':

     8,432,416,866      cycles                    #    2.886 GHz                      
     7,114,758,602      instructions              #    0.84  insn per cycle           
     1,040,831,469      cache-references          #  356.176 M/sec                    
       216,490,683      cache-misses              #   20.800 % of all cache refs      
     1,252,928,847      branches                  #  428.756 M/sec                    
         8,174,531      branch-misses             #    0.65% of all branches          
       2922.239426      task-clock (msec)         #    1.285 CPUs utilized          
           403,282      faults                    #    0.138 M/sec                  
           403,282      minor-faults              #    0.138 M/sec                  
                96      cs                        #    0.033 K/sec                  
                 5      migrations                #    0.002 K/sec                  

       2.274377105 seconds time elapsed

* [예제 6-11] 벡터 연산 기능을 제회한 numpy 2차원 확산 방정식의 성능

In [ ]:
!perf stat -e cycles,instructions,\
    cache-references,cache-misses,branches,brance-misses,task-clock,faults,\
    minor-faults,cs,migrations python diffusion_numpy.py

 Performance counter stats for 'python diffusion_numpy.py':

    50,086,999,350      cycles                    #    2.888 GHz                      
    53,611,608,977      instructions              #    1.07  insn per cycle           
     1,131,742,674      cache-references          #   65.266 M/sec                    
       322,483,897      cache-misses              #   28.494 % of all cache refs      
     4,001,923,035      branches                  #  230.785 M/sec                    
         6,211,101      branch-misses             #    0.16% of all branches          
      17340.464580      task-clock (msec)         #    1.000 CPUs utilized          
           403,193      faults                    #    0.023 M/sec                  
           403,193      minor-faults              #    0.023 M/sec                  
                74      cs                        #    0.004 K/sec                  
                 6      migrations                #    0.000 K/sec                  

      17.339656586 seconds time elapsed

해석 : 속도 개선은 벡터 연산 때문이라기보다 메모리 단편화의 감소와 메모리 지역성 덕분이라고 볼 수 있다.

&nbsp;
&nbsp;
&nbsp;
&nbsp;
&nbsp;

6.4.1 메모리 할당과 제자리 연산
----------------------------

* [예제 6-12] 메모리 할당을 줄이기 위한 제자리 연산

In [1]:
import numpy as np
array1 = np.random.random((10,10))
array2 = np.random.random((10,10))

print(id(array1))

array1 += array2
print(id(array1))

array1 = array1 + array2       # ----> 메모리 주소가 바뀜
print(id(array1))

139691745065744
139691745065744
139691745066320


In [9]:
%%timeit array1,array2 = np.random.random((2,100,100))
array1 = array1 + array2

5.51 µs ± 31.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
%%timeit array1,array2 = np.random.random((2,100,100))
array1 += array2

4.54 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
%%timeit array1,array2 = np.random.random((2,5,5))
array1 = array1 + array2

461 ns ± 3.36 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [12]:
%%timeit array1,array2 = np.random.random((2,5,5))
array1 += array2

1.06 µs ± 0.662 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


해석 : 제자리 연산은 메모리 할당을 할 필요가 없으므로 속도가 더 빠르다    
다만 배열 크기가 CPU캐십다 큰 경우에만 이점이 있다. 배열 크기가 작으면 그냥 하는게 더 낫다

* [예제 6-14] numpy연산을 제자리 연산으로 변경

In [ ]:
import time
from numpy import (zeros,roll)

grid_shape = (640,640)

def laplacian(grid,out):
    np.copyto(out,grid)
    out *= -4
    out += np.roll(grid,+1,0)
    out += np.roll(grid,-1,0)
    out += np.roll(grid,+1,1)
    out += np.roll(grid,-1,1)

def evolve(grid,dt,out,D=1):
    laplacia(grid,out)
    out *= D*dt
    out += grid

def run_experiment(num_iterations):
    next_grid = np.zeros(grid_shape)
    grid = zeros(grid_shape)
    
    block_row = int(grid_shape[0]*0.4)
    block_high = int(grid_shape[0]*0.5)
    grid[block_row:block_high,block_row:block_high] = 0.005
    
    start = time.time()
    for i in range(num_iterations):
        evolve(grid,0.1,next_grid)
        grid,next_grid = next_grid,grid
        
    return time.time() - start

In [ ]:
!perf stat -e cycles,instructions,\
    cache-references,cache-misses,branches,brance-misses,task-clock,faults,\
    minor-faults,cs,migrations python diffusion_numpy_memory.py


 Performance counter stats for 'python diffusion_numpy_memory.py':

     6,880,906,446      cycles                    #    2.886 GHz                      
     5,848,134,537      instructions              #    0.85  insn per cycle           
     1,077,550,720      cache-references          #  452.000 M/sec                    
       217,974,413      cache-misses              #   20.229 % of all cache refs      
     1,028,769,315      branches                  #  431.538 M/sec                    
         7,492,245      branch-misses             #    0.73% of all branches          
       2383.962679      task-clock (msec)         #    1.373 CPUs utilized          
            13,521      faults                    #    0.006 M/sec                  
            13,521      minor-faults              #    0.006 M/sec                  
               100      cs                        #    0.042 K/sec                  
                 8      migrations                #    0.003 K/sec                  

       1.736322099 seconds time elapsed



In [ ]:
이후 내용은 포기..... 너무 딥러닝과는 거리가 멀고 너무 깊다